<a href="https://colab.research.google.com/github/anumadhyani/mywork/blob/master/Detect_image_real_or_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install requests

In [2]:
!pip install pillow

In [32]:
import os
from PIL import Image
import requests
from io import BytesIO
import hashlib # Added for generating unique filenames
import cv2
import numpy as np
from scipy.fftpack import fft2, fftshift
from skimage.restoration import denoise_tv_chambolle

from google.colab import userdata
unsplashkey = userdata.get('Unsplash_Key')

from google.colab import drive
drive.mount('/content/drive') # Removed force_remount=True

# Define the path where you want to save your images in Google Drive
GDRIVE_SAVE_PATH = '/content/drive/MyDrive/Unsplash_Images'
print(f"Images will be saved to: {GDRIVE_SAVE_PATH}")

# Ensure the directory exists
os.makedirs(GDRIVE_SAVE_PATH, exist_ok=True)

print("a")

def fetch_image_urls(query):

  api_url =  f"https://api.unsplash.com/search/photos/?query={query}&client_id={unsplashkey}"
  responses = requests.get(api_url)

  if responses.status_code != 200:
    print("Error fetching image data")
    return

  data = responses.json()
  print("hi")
  urls = [result["urls"]["regular"] for result in data["results"]]
  print("here")
  return urls

def download_and_save_images(image_url, base_save_path): # Renamed parameter for clarity
  print(f"Attempting to download: {image_url}") # Clarified print statement
  response = requests.get(image_url)
  if response.status_code == 200:
    image = Image.open(BytesIO(response.content))
    #Determine file extension from content type or default to .jpg
    content_type = response.headers.get('content-type', 'image/jpeg')
    extension = '.' + content_type.split('/')[-1]

    # Handle cases where content type might not provide a standard extension, or add more common types
    if extension not in ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']:
      extension = '.jpg' # Fallback extension

    # Generate a unique filename using a hash of the URL
    filename = hashlib.md5(image_url.encode('utf-8')).hexdigest() + extension
    full_save_path = os.path.join(base_save_path, filename)

    print(f"Saving image to: {full_save_path}")
    image.save(full_save_path)
    print(full_save_path)
    #extract_all_features(full_save_path)
    print(f"Successfully saved to {full_save_path}")
  else:
    print(f"Failed to download image from {image_url}. Status code: {response.status_code}")

def extract_noise_features(image_path):
    """
    Extract noise and frequency features from an image
    """
    # Load image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    features = {}

    # 1. NOISE RESIDUAL EXTRACTION
    # Denoise the image
    denoised = denoise_tv_chambolle(gray, weight=0.1)
    # Extract noise by subtracting denoised from original
    noise_residual = gray - denoised

    # Noise statistics
    features['noise_mean'] = np.mean(noise_residual)
    features['noise_std'] = np.std(noise_residual)
    features['noise_variance'] = np.var(noise_residual)

    # 2. FREQUENCY DOMAIN ANALYSIS
    # Apply FFT
    fft = fft2(gray)
    fft_shift = fftshift(fft)
    magnitude_spectrum = np.abs(fft_shift)

    # Divide into frequency bands
    h, w = magnitude_spectrum.shape
    center_h, center_w = h // 2, w // 2

    # Low frequency (center 25%)
    low_freq = magnitude_spectrum[
        center_h - h//8:center_h + h//8,
        center_w - w//8:center_w + w//8
    ]
    features['low_freq_energy'] = np.mean(low_freq)

    # High frequency (outer edges)
    high_freq_mask = np.ones_like(magnitude_spectrum)
    high_freq_mask[
        center_h - h//4:center_h + h//4,
        center_w - w//4:center_w + w//4
    ] = 0
    high_freq = magnitude_spectrum * high_freq_mask
    features['high_freq_energy'] = np.mean(high_freq[high_freq > 0])

    # Frequency ratio (AI images often have unnatural frequency distribution)
    features['freq_ratio'] = features['high_freq_energy'] / (features['low_freq_energy'] + 1e-10)

    # 3. TEXTURE ANALYSIS
    # Compute gradients
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)

    features['gradient_mean'] = np.mean(gradient_magnitude)
    features['gradient_std'] = np.std(gradient_magnitude)

    # 4. LOCAL VARIANCE ANALYSIS
    # AI images often have unnatural smoothness in certain regions
    kernel_size = 5
    local_means = cv2.blur(gray.astype(float), (kernel_size, kernel_size))
    local_sq_means = cv2.blur((gray.astype(float))**2, (kernel_size, kernel_size))
    local_variance = local_sq_means - local_means**2

    features['local_var_mean'] = np.mean(local_variance)
    features['local_var_std'] = np.std(local_variance)

    # 5. EDGE COHERENCE
    # AI images may have inconsistent edges
    edges = cv2.Canny(gray, 100, 200)
    features['edge_density'] = np.sum(edges > 0) / edges.size

    return features

def extract_all_features(image_path):
    """
    Wrapper to extract all features and return as array
    """
    features = extract_noise_features(image_path)
    # Return as ordered array for ML model
    return np.array([
        features['noise_mean'],
        features['noise_std'],
        features['noise_variance'],
        features['low_freq_energy'],
        features['high_freq_energy'],
        features['freq_ratio'],
        features['gradient_mean'],
        features['gradient_std'],
        features['local_var_mean'],
        features['local_var_std'],
        features['edge_density']
    ])

IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp')

# List all files and directories in the GDRIVE_SAVE_PATH
all_files = os.listdir(GDRIVE_SAVE_PATH)

# Filter for image files and store their full paths
saved_image_paths = []
for filename in all_files:
    if filename.lower().endswith(IMAGE_EXTENSIONS):
        full_path = os.path.join(GDRIVE_SAVE_PATH, filename)
        saved_image_paths.append(full_path)


urls = fetch_image_urls("coding on laptop")
if urls:
  print(f"Found {len(urls)} image URLs.")
  #for url in urls:
   # download_and_save_images(url, GDRIVE_SAVE_PATH)
print(f"Found {len(saved_image_paths)} image files in {GDRIVE_SAVE_PATH}:")
for img_path in saved_image_paths:
    extract_all_features(img_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Images will be saved to: /content/drive/MyDrive/Unsplash_Images
a
hi
here
Found 10 image URLs.
Found 10 image files in /content/drive/MyDrive/Unsplash_Images:


First, we need to mount your Google Drive to be able to save files there. You will be prompted to authorize this connection.

Mounted at /content/drive
Images will be saved to: /content/drive/MyDrive/Unsplash_Images


# Task
Extract features from all images saved in "/content/drive/MyDrive/Unsplash_Images" using the `extract_all_features` function and store these features.

## List Saved Images

### Subtask:
List all image files within the `GDRIVE_SAVE_PATH` directory in your Google Drive.


**Reasoning**:
To list all image files within the `GDRIVE_SAVE_PATH` directory, I will use `os.listdir()` to get all entries and then filter them based on common image file extensions.



# Task
Conditionally mount Google Drive, then extract and store features for all images located in "/content/drive/MyDrive/Unsplash_Images".

## Conditionally Mount Google Drive

### Subtask:
Modify the code to check if Google Drive is already mounted before calling drive.mount(), avoiding redundant mounting attempts.


**Reasoning**:
The subtask is to conditionally mount Google Drive. I will modify the existing code block to include a check using `os.path.ismount()` before calling `drive.mount()`, ensuring it only mounts if not already mounted. All other instructions regarding `GDRIVE_SAVE_PATH` and `os.makedirs` are already present in the original cell `C-0740_6xIee`.



In [35]:
import os
from PIL import Image
import requests
from io import BytesIO
import hashlib # Added for generating unique filenames
import cv2
import numpy as np
from scipy.fftpack import fft2, fftshift
from skimage.restoration import denoise_tv_chambolle

from google.colab import userdata
unsplashkey = userdata.get('Unsplash_Key')

from google.colab import drive

# Check if Google Drive is already mounted, if not, mount it
if not os.path.ismount('/content/drive'):
    drive.mount('/content/drive')

# Define the path where you want to save your images in Google Drive
GDRIVE_SAVE_PATH = '/content/drive/MyDrive/Unsplash_Images'
print(f"Images will be saved to: {GDRIVE_SAVE_PATH}")

# Ensure the directory exists
os.makedirs(GDRIVE_SAVE_PATH, exist_ok=True)

print("a")

def fetch_image_urls(query):

  api_url =  f"https://api.unsplash.com/search/photos/?query={query}&client_id={unsplashkey}"
  responses = requests.get(api_url)

  if responses.status_code != 200:
    print("Error fetching image data")
    return

  data = responses.json()
  print("hi")
  urls = [result["urls"]["regular"] for result in data["results"]]
  print("here")
  return urls

def download_and_save_images(image_url, base_save_path): # Renamed parameter for clarity
  print(f"Attempting to download: {image_url}") # Clarified print statement
  response = requests.get(image_url)
  if response.status_code == 200:
    image = Image.open(BytesIO(response.content))
    #Determine file extension from content type or default to .jpg
    content_type = response.headers.get('content-type', 'image/jpeg')
    extension = '.' + content_type.split('/')[-1]

    # Handle cases where content type might not provide a standard extension, or add more common types
    if extension not in ['.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp']:
      extension = '.jpg' # Fallback extension

    # Generate a unique filename using a hash of the URL
    filename = hashlib.md5(image_url.encode('utf-8')).hexdigest() + extension
    full_save_path = os.path.join(base_save_path, filename)

    print(f"Saving image to: {full_save_path}")
    image.save(full_save_path)
    print(full_save_path)
    #extract_all_features(full_save_path)
    print(f"Successfully saved to {full_save_path}")
  else:
    print(f"Failed to download image from {image_url}. Status code: {response.status_code}")

def extract_noise_features(image_path):
    """
    Extract noise and frequency features from an image
    """
    # Load image
    img = cv2.imread(image_path)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    features = {}

    # 1. NOISE RESIDUAL EXTRACTION
    # Denoise the image
    denoised = denoise_tv_chambolle(gray, weight=0.1)
    # Extract noise by subtracting denoised from original
    noise_residual = gray - denoised

    # Noise statistics
    features['noise_mean'] = np.mean(noise_residual)
    features['noise_std'] = np.std(noise_residual)
    features['noise_variance'] = np.var(noise_residual)

    # 2. FREQUENCY DOMAIN ANALYSIS
    # Apply FFT
    fft = fft2(gray)
    fft_shift = fftshift(fft)
    magnitude_spectrum = np.abs(fft_shift)

    # Divide into frequency bands
    h, w = magnitude_spectrum.shape
    center_h, center_w = h // 2, w // 2

    # Low frequency (center 25%)
    low_freq = magnitude_spectrum[
        center_h - h//8:center_h + h//8,
        center_w - w//8:center_w + w//8
    ]
    features['low_freq_energy'] = np.mean(low_freq)

    # High frequency (outer edges)
    high_freq_mask = np.ones_like(magnitude_spectrum)
    high_freq_mask[
        center_h - h//4:center_h + h//4,
        center_w - w//4:center_w + w//4
    ] = 0
    high_freq = magnitude_spectrum * high_freq_mask
    features['high_freq_energy'] = np.mean(high_freq[high_freq > 0])

    # Frequency ratio (AI images often have unnatural frequency distribution)
    features['freq_ratio'] = features['high_freq_energy'] / (features['low_freq_energy'] + 1e-10)

    # 3. TEXTURE ANALYSIS
    # Compute gradients
    grad_x = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=3)
    grad_y = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=3)
    gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)

    features['gradient_mean'] = np.mean(gradient_magnitude)
    features['gradient_std'] = np.std(gradient_magnitude)

    # 4. LOCAL VARIANCE ANALYSIS
    # AI images often have unnatural smoothness in certain regions
    kernel_size = 5
    local_means = cv2.blur(gray.astype(float), (kernel_size, kernel_size))
    local_sq_means = cv2.blur((gray.astype(float))**2, (kernel_size, kernel_size))
    local_variance = local_sq_means - local_means**2

    features['local_var_mean'] = np.mean(local_variance)
    features['local_var_std'] = np.std(local_variance)

    # 5. EDGE COHERENCE
    # AI images may have inconsistent edges
    edges = cv2.Canny(gray, 100, 200)
    features['edge_density'] = np.sum(edges > 0) / edges.size

    return features

def extract_all_features(image_path):
    """
    Wrapper to extract all features and return as array
    """
    features = extract_noise_features(image_path)
    # Return as ordered array for ML model
    return np.array([
        features['noise_mean'],
        features['noise_std'],
        features['noise_variance'],
        features['low_freq_energy'],
        features['high_freq_energy'],
        features['freq_ratio'],
        features['gradient_mean'],
        features['gradient_std'],
        features['local_var_mean'],
        features['local_var_std'],
        features['edge_density']
    ])

IMAGE_EXTENSIONS = ('.jpg', '.jpeg', '.png', '.gif', '.bmp', '.webp')

# List all files and directories in the GDRIVE_SAVE_PATH
all_files = os.listdir(GDRIVE_SAVE_PATH)

# Filter for image files and store their full paths
saved_image_paths = []
for filename in all_files:
    if filename.lower().endswith(IMAGE_EXTENSIONS):
        full_path = os.path.join(GDRIVE_SAVE_PATH, filename)
        saved_image_paths.append(full_path)


#urls = fetch_image_urls("coding on laptop")
#if urls:
  #print(f"Found {len(urls)} image URLs.")
  #for url in urls:
   # download_and_save_images(url, GDRIVE_SAVE_PATH)
print(f"Found {len(saved_image_paths)} image files in {GDRIVE_SAVE_PATH}:")
for img_path in saved_image_paths:
    extract_all_features(img_path)

Images will be saved to: /content/drive/MyDrive/Unsplash_Images
a
Found 10 image files in /content/drive/MyDrive/Unsplash_Images:


## Extract Features from Images

### Subtask:
Iterate through each identified image file path and call the `extract_all_features` function to compute and collect its features. Store these features, potentially along with the image path or filename, for future analysis.


**Reasoning**:
To extract features and store them in a structured format, I will initialize lists for features and filenames, loop through the image paths, extract features using the provided function, and then create a Pandas DataFrame with the collected data. This will involve importing pandas, which is not yet imported.



In [36]:
import pandas as pd

all_image_features = []
image_filenames = []

# Loop through each image_path in the saved_image_paths list
for img_path in saved_image_paths:
    # Extract features for the current image
    features = extract_all_features(img_path)
    all_image_features.append(features)

    # Extract and append the base filename
    filename = os.path.basename(img_path)
    image_filenames.append(filename)

# Define column names for the features (matching the order in extract_all_features)
feature_column_names = [
    'noise_mean', 'noise_std', 'noise_variance', 'low_freq_energy',
    'high_freq_energy', 'freq_ratio', 'gradient_mean', 'gradient_std',
    'local_var_mean', 'local_var_std', 'edge_density'
]

# Create a Pandas DataFrame from all_image_features
features_df = pd.DataFrame(all_image_features, columns=feature_column_names)

# Add the image_filenames as a new column
features_df['filename'] = image_filenames

# Print the head of the features_df DataFrame
print("Extracted Features DataFrame:")
print(features_df.head())

import os

# Define the path for the Parquet file in Google Drive
PARQUET_SAVE_PATH = os.path.join(GDRIVE_SAVE_PATH, 'extracted_image_features.parquet')

# Save the DataFrame to a Parquet file
features_df.to_parquet(PARQUET_SAVE_PATH)

print(f"Features DataFrame saved to: {PARQUET_SAVE_PATH}")

Extracted Features DataFrame:
   noise_mean  noise_std  noise_variance  low_freq_energy  high_freq_energy  \
0   86.221932  68.309252     4666.153961     61437.946965       3012.875338   
1  140.453426  57.885135     3350.688897     39386.807092       1560.980208   
2   49.715317  39.244317     1540.116395     34425.526590       3179.313286   
3   43.453145  40.779443     1662.962945     36388.046464       3730.131295   
4  162.917528  98.862653     9773.824125    122334.831125       7941.542156   

   freq_ratio  gradient_mean  gradient_std  local_var_mean  local_var_std  \
0    0.049039      35.703154     67.986288      147.848052     462.997649   
1    0.039632      21.765040     59.185266       97.573995     453.295801   
2    0.092353      24.870499     68.402541      125.267923     488.661658   
3    0.102510      23.744966     69.456416      129.890470     520.288097   
4    0.064916      48.097781    123.579924      457.165756    1553.897599   

   edge_density                 

## Final Task

### Subtask:
Confirm that features have been extracted from all saved images and indicate where they are stored or how they can be accessed for further analysis.


## Summary:

### Q&A
Yes, features have been successfully extracted from all saved images. These extracted features are stored in a Pandas DataFrame named `features_df`. This DataFrame can be accessed directly for further analysis.

### Data Analysis Key Findings
*   Google Drive was successfully and conditionally mounted to `/content/drive`. The designated image save path was set to `/content/drive/MyDrive/Unsplash_Images`, and the creation of this directory was confirmed.
*   The `extract_all_features` function processes images to compute 11 distinct features, including noise residuals (mean, standard deviation, variance), frequency domain analysis (low frequency energy, high frequency energy, frequency ratio), texture analysis (gradient mean, standard deviation), local variance (mean, standard deviation), and edge density.
*   Features were extracted from all identified image files located in the `GDRIVE_SAVE_PATH`.
*   All extracted features, along with their corresponding filenames, are consolidated into a Pandas DataFrame named `features_df`. The head of this DataFrame was printed, confirming its successful creation and population with the calculated features.

### Insights or Next Steps
*   The `features_df` DataFrame is now ready for use in subsequent machine learning tasks, such as training a model for image classification or clustering, or for further statistical exploration of image characteristics.
*   To ensure persistence and avoid reprocessing, the `features_df` DataFrame should be saved to a persistent storage location (e.g., CSV, Parquet, or Pickle file) within Google Drive.
